In [67]:
import pandas as pd
import openpyxl

# Load the data
df = pd.read_csv('PRG Sales Comparables Table.csv', encoding='latin1')

# Display unique counties and their corresponding indices
unique_counties = df['County'].unique()
print("Select a county by number:")
for index, county in enumerate(unique_counties):
    print(f"{index}: {county}")

# Get user input for county
try:
    user_input = int(input("Enter the number corresponding to the county: "))
    
    # Check if the input is valid
    if 0 <= user_input < len(unique_counties):
        selected_county = unique_counties[user_input]
        df_filtered_by_county = df[df['County'] == selected_county]  # Filter DataFrame by county
        
        print(f"\nFiltered DataFrame for county: {selected_county}")

        # Display unique uses for the selected county
        unique_uses = df_filtered_by_county['Use'].unique()
        print("\nSelect a use by number:")
        for index, use in enumerate(unique_uses):
            print(f"{index}: {use}")

        # Get user input for use
        try:
            user_input = int(input("Enter the number corresponding to the use: "))
            
            # Check if the input is valid
            if 0 <= user_input < len(unique_uses):
                selected_use = unique_uses[user_input]
                df = df_filtered_by_county[df_filtered_by_county['Use'] == selected_use]  # Filter DataFrame by use
                
                print(f"\nFiltered DataFrame for use: {selected_use}")
            else:
                print("Invalid selection. Please select a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")
            
    else:
        print("Invalid selection. Please select a number from the list.")
except ValueError:
    print("Invalid input. Please enter a number.")

df = df.head(5)

# Load the Excel file
file_path = 'flex_comp_grid.xlsx'
workbook = openpyxl.load_workbook(file_path)
sheet = workbook.active

# Display the contents of the third column for reference
third_column_data = [row[0] for row in sheet.iter_rows(min_col=3, max_col=3, values_only=True)]
print("Original third column data:", third_column_data)

num_duplicates = df.shape[0]

# Start processing each row
for row in range(1, len(third_column_data) + 1):  # Loop through all rows of the third column
    original_cell = sheet.cell(row=row, column=3)  # Original cell in the third column

    # Move the existing values from the fourth column to the right for all rows
    for j in range(num_duplicates):
        existing_cell = sheet.cell(row=row, column=4)  # Cell in the fourth column
        new_cell = sheet.cell(row=row, column=4 + j)  # Move existing values to the right
        new_cell.value = existing_cell.value  # Copy the existing value to the new cell

        # Copy formatting attributes from the original fourth column cell
        if existing_cell.has_style:
            new_cell.font = existing_cell.font.copy()
            new_cell.fill = existing_cell.fill.copy()
            new_cell.border = existing_cell.border.copy()
            new_cell.alignment = existing_cell.alignment.copy()
            new_cell.number_format = existing_cell.number_format

    # Duplicating values in the third column
    if row == 4:  # For the fourth row, increment the values
        for i in range(num_duplicates):
            new_cell = sheet.cell(row=row, column=3 + i)  # Place duplicates in the correct column
            new_cell.value = original_cell.value + i  # Incremented value for each duplicate
            # Copy formatting attributes
            if original_cell.has_style:
                new_cell.font = original_cell.font.copy()
                new_cell.fill = original_cell.fill.copy()
                new_cell.border = original_cell.border.copy()
                new_cell.alignment = original_cell.alignment.copy()
                new_cell.number_format = original_cell.number_format
    else:  # For all other rows, just duplicate the original value
        for i in range(num_duplicates):
            new_cell = sheet.cell(row=row, column=3 + i)  # Place duplicates in the correct column
            new_cell.value = original_cell.value  # Copy the original value
            # Copy formatting attributes
            if original_cell.has_style:
                new_cell.font = original_cell.font.copy()
                new_cell.fill = original_cell.fill.copy()
                new_cell.border = original_cell.border.copy()
                new_cell.alignment = original_cell.alignment.copy()
                new_cell.number_format = original_cell.number_format

# Save the modified workbook to a new Excel file
output_file_path = 'duplicated_columns_with_incremented_fourth_row.xlsx'
workbook.save(output_file_path)

print(f"Duplicated columns with incremented fourth row saved to '{output_file_path}'.")



Select a county by number:
0: Allegany
1: Carroll
2: Carroll 
3: Charles
4: Frederick
5: Montgomery
6: Prince George
7: Washington
8: Garrett
9: Baltimore
10: Anne Arundel
11: Saint Mary's
12: Howard
13: Calvert
14: Washington 
15: Frederick 
16: Jefferson


Enter the number corresponding to the county:  4



Filtered DataFrame for county: Frederick

Select a use by number:
0: Auto Sales/ Service
1: Medical Office
2: Mixed
3: Office Condo
4: Professional Office
5: Retail
6: Vacant Land
7: Warehouse Condo
8: Warehouse/ Flex
9: Restaurant
10: Church
11: Mixed 
12: Apartment
13: Daycare
14: Special Purpose
15: Convenience Store
16: Warehouse


Enter the number corresponding to the use:  2



Filtered DataFrame for use: Mixed
Original third column data: [None, None, None, 1, None, None, None, None, None, '=C7/C9', None, None, None, None, '=C6/C7', None, 'FEE SIMPLE', 0, '=C15*C18+C15', 'TYPICAL', 0, '=C19*C21+C19', 'ARMS-LENGTH', 0, '=C22*C24+C22', 'YES', 0.08, '=C25*C27+C25', 'STABLE', 0, '=C28*C30+C28', None, 0, 0, 0, 0, 0, 0, 0, '=SUM(C33:C39)', '=C31*C40+C31']
Duplicated columns with incremented fourth row saved to 'duplicated_columns_with_incremented_fourth_row.xlsx'.


C:\Users\micha\AppData\Local\Temp\ipykernel_21948\2975912448.py:75: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.font = existing_cell.font.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_21948\2975912448.py:76: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.fill = existing_cell.fill.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_21948\2975912448.py:77: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.border = existing_cell.border.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_21948\2975912448.py:78: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.alignment = existing_cell.alignment.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_21948\2975912448.py:99: DeprecationWarning: Call to deprecated function copy (Use copy(obj